<a href="https://colab.research.google.com/github/DorraBoutiti/InnovIT/blob/main/Copie_de_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import numpy

import tflearn
import tensorflow
import random
import json
import pickle


In [ ]:

with open("intents.json") as file:
  data = json.load(file)
try:
  with open("data.pickle","rb") as f:
    words, labels, training, output = pickle.load(f)
except:
  words = []
  labels = []
  docs_x = []
  docs_y = []
  for intent in data["intents"]:
    for pattern in intent["patterns"]:
      wrds = nltk.word_tokenize(pattern)
      words.extend(wrds)
      docs_x.append(wrds)
      docs_y.append(intent["tag"])
      
      if intent["tag"] not in labels:
        labels.append(intent["tag"])
    words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
    words = sorted(list(set(words)))
    labels = sorted(labels)
    training = []
    output = []
    out_empty = [0 for _ in range(len(labels))]
    for x, doc in enumerate(docs_x):
      bag = []
      wrds = [stemmer.stem(w) for w in doc]
      for w in words:
        if w in wrds:
          bag.append(1)
        else:
          bag.append(0)
      output_row = out_empty[:]
      output_row[labels.index(docs_y[x])] = 1
      training.append(bag)
      output.append(output_row)
    training = numpy.array(training)
    output = numpy.array(output)
    with open("data.pickle","wb") as f:
      pickle.dump((words, labels, training, output),f)



# Nouvelle section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")


Training Step: 15999  | total loss: 0.01880 | time: 0.081s
| Adam | epoch: 1000 | loss: 0.01880 - acc: 0.9931 -- iter: 120/121
Training Step: 16000  | total loss: 0.02569 | time: 0.085s
| Adam | epoch: 1000 | loss: 0.02569 - acc: 0.9938 -- iter: 121/121
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
def bag_of_words(s,words):
  bag = [0 for _ in range(len(words))]

  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  for se in s_words:
    for i, w in enumerate(words):
      if w == se:
        bag[i] = 1
  return(numpy.array(bag))

In [ ]:
def chat():
  print("Start talking with the Bot(type quit to stop)!")
  while True:
    inp = input("You: ")
    if inp.lower() == "quit":
      break
    results = model.predict([bag_of_words(inp, words)])
    results_index = numpy.argmax(results)
    tag = labels[results_index]
    for tg in data["intents"]:
      if tg['tag'] == tag:
        responses = tg['responses']
    print(random.choice(responses))
    
     

In [ ]:
chat()

Start talking with the Bot(type quit to stop)!
You: Hello
Good to see you again
You: How much?
Can't Understand Your Question
You: How much it costs?
Yes
You: 
please elaborate your question
You: how you could deliver me?
Please enter your email id ,we will send a link on your email
You: abc@gmail.com
Link has been shared.Please change your password
You: I want to change my appointment
The nearest manufacturer is Vietnam
You: What are events?
Recent news of Demonetisation & recession
You: how is the market?
It's 36C according to accuweather
You: I want to meet Khaled
I will just check if he is available or on leave.
You: I have a problem
Thanks For Asking!How can I help you?
You: I miss Dorra
please elaborate your question
You: problem?
Hello, thanks for asking
You: Impacts on sale?
this year sale might be impacted due to demonitisation
You: improve sale?
Production Schedule,Delivery Schedule,Proxy information about cost
You: the customer response?
Customer was happy and has given good